## LSTM sample

In [ ]:
symbol = "AAP"

### imports

In [ ]:
from datetime import datetime, date, timedelta
import alpaca_trade_api as tradeapi
from typing import List, Tuple
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

### Load data

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
ohlc_data = api.polygon.historic_agg_v2(
    symbol,
    1,
    "minute",
    _from=str(date.today() - timedelta(days=7)),
    to=str(date.today()),
).df

In [ ]:
x_dates = ohlc_data.index
ohlc_data['date'] = np.arange(start = 0, stop = len(ohlc_data), step = 1, dtype='int')
ohlc_data=ohlc_data.set_index(ohlc_data.date)

In [ ]:
ohlc_data.close.plot()
plt.show()

### Pre-processing

In [ ]:
ohlc_data["delta"] = ohlc_data.close.diff()
ohlc_data["delta_pct"] = ohlc_data.close.pct_change()

In [ ]:
ohlc_data.delta.plot()
plt.show()
ohlc_data.delta_pct.plot()
plt.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
ohlc_data.delta = scaler.fit_transform(ohlc_data.delta.values.reshape(-1, 1)).flatten()

plt.figure(figsize=(10, 4))
ohlc_data.delta.plot()

In [ ]:
ohlc_data.shape

In [ ]:
def wind(df, windows):
    train_cols = []
    for window in reversed(windows):    
        df['shift_pos_' + str(window)] = df.delta.shift(window).fillna(0)
        train_cols.append('shift_pos_' + str(window))
    return df[len(windows):], train_cols

In [ ]:
df, train_cols = wind(ohlc_data, range(1, 15))
print(df.shape)

In [ ]:
g = 0
l = 8
c = 0
for i, row in df.iterrows():
    df.loc[df.index==i, 'group'] = int(g)
    if c == l-1:
        c = 0
        g += 1
    else:
        c += 1

In [ ]:
test_cols = ['delta']
df.shape

In [ ]:
train_df = df.iloc[:-(2*l), :]
test_df = df.iloc[-(2*l):, :]
train_df.shape, test_df.shape

In [ ]:
X_train = np.array(list(train_df.groupby('group').apply(lambda x: x[train_cols].values))).astype(np.float32)
y_train = np.array(list(train_df.groupby('group').apply(lambda x: x[test_cols].values))).astype(np.float32)
X_train.shape, y_train.shape


In [ ]:
plt.plot(y_train[-4:].flatten())

In [ ]:
series = original_data
sample_len = 4
len(series)

In [ ]:
dataset : List[Tuple] = []

In [ ]:
dataset = [[x for x in series[ind:ind+sample_len]] for ind in range(len(series)-sample_len)] 
len(dataset)

## Training

In [ ]:
torch.manual_seed(1)
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)
